Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# Deploy a model using Prebuilt Docker Images for Inference

In this tutorial, you will deploy a PyTorch model using the Azure Machine Learning Python SDK. The PyTorch model is trained to classify chickens and turkeys by first using a pretrained ResNet18 model that has been trained on the [ImageNet](http://image-net.org/index) dataset. 


## Prerequisites

If you are using an Azure Machine Learning Notebook VM, you are all set. Otherwise, go through the [Configuration](../../../../configuration.ipynb) notebook to install the Azure Machine Learning Python SDK and create an Azure ML `Workspace`

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize workspace
Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`.

In [ ]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print(
    "Workspace name: " + ws.name,
    "Azure region: " + ws.location,
    "Subscription id: " + ws.subscription_id,
    "Resource group: " + ws.resource_group,
    sep="\n",
)

## Register model
Register a file or folder as a model by calling [Model.register()](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py#register-workspace--model-path--model-name--tags-none--properties-none--description-none--datasets-none--model-framework-none--model-framework-version-none--child-paths-none--sample-input-dataset-none--sample-output-dataset-none--resource-configuration-none-)

In addition to the content of the model file itself, your registered model will also store model metadata -- model description, tags, and framework information -- that will be useful when managing and deploying models in your workspace. Using tags, for instance, you can categorize your models and apply filters when listing models in your workspace

In [ ]:
from azureml.core.model import Model

# Register the model
model = Model.register(
    model_name="pytorch-birds-model",
    model_path="pytorch-birds/outputs/model.pt",
    workspace=ws,
)

## Deploy model as web service
Once you have your trained model, you can deploy the model on Azure. 

In this tutorial, we will deploy the model as a web service in [Azure Container Instances (ACI)](https://docs.microsoft.com/en-us/azure/container-instances/).

For more information on deploying models using Azure Machine Learning, refer [here](https://docs.microsoft.com/azure/machine-learning/service/how-to-deploy-and-where).

# Write the requirements.txt
Write the `requirements.txt`

> **Pin each package version:**
Even while prototyping, pin each package version in requirements.txt. 
For example, use scipy == 1.2.3 instead of just scipy or even scipy > 1.2.3. 
If you don't pin an exact version and scipy releases a new version, this can break your scoring script and cause failures during deployment and scaling.

In [ ]:
%%writefile requirements.txt
  torchvision==0.7.0
  future==0.17.1
  pillow==8.1.1
  azureml.core==1.25.0

## Define the environment (Dynamic installation via requirements.txt)

Then, we will need to create an Azure Machine Learning environment that specifies all of the scoring script's package dependencies. 

Here we assume the `requirements.txt` is in the right format and inlcudes all additional packages for the model.

If deployment fails because of Python extensibility solution, there will be no logs available the [Azure Machine Learning Studio](https://ml.azure.com),`service.get_logs()` will return None. If there is a problem in the *init()* function of score.py, `service.get_logs()` will return logs for the same.

For local debugging and validation, run the following command in `score.py` folder :

```
docker run -it -v $(pwd):/var/azureml-app -e AZUREML_EXTRA_REQUIREMENTS_TXT="requirements.txt" mcr.microsoft.com/azureml/pytorch-1.6-ubuntu18.04-py37-cpu-inference:latest 
```

In [ ]:
from azureml.core import Environment
from azureml.core.conda_dependencies import CondaDependencies

myenv = Environment(name="my_pytroch_env")
myenv.docker.enabled = True
myenv.docker.base_image = (
    "mcr.microsoft.com/azureml/pytorch-1.6-ubuntu18.04-py37-cpu-inference:latest "
)
myenv.python.user_managed_dependencies = True
myenv.register(workspace=ws)

myenv.environment_variables = {"AZUREML_EXTRA_REQUIREMENTS_TXT": "requirements.txt"}

### Create scoring script

First, we will create a scoring script that will be invoked by the web service call. Note that the scoring script must have two required functions:
* `init()`: In this function, you typically load the model into a `global` object. This function is executed only once when the Docker container is started. 
* `run(input_data)`: In this function, the model is used to predict a value based on the input data. The input and output typically use JSON as serialization and deserialization format, but you are not limited to that.

Refer to the scoring script `pytorch_score.py` for this tutorial. Our web service will use this file to predict whether an image is a chicken or a turkey. When writing your own scoring script, don't forget to test it locally first before you go and deploy the web service.

## Use Azure Machine Learning Inference HTTP Server before deployment

The local inference server allows users to quickly debug their score script. In the case that the underlying score script has a bug, the server will fail to initialize/serve and will instead throw an exception & LOC where the issues occurred at. It enables users to easily create cohesive validation gates in their CI/CD pipelines. In order to do so, simple boot up the server with the candidate script and run the test suite against the local endpoint.

**How to use the Inference HTTP Server?**

* Install the ***azureml-inference-server-http*** package

```sh
python -m pip install azureml-inference-server-http
```

* Start the server and set score.py as the entry script.
```sh
azmlinfsrv --entry_script score.py
```

* Let's send a scoring request to the server using curl.

```sh
curl -p 127.0.0.1:5001/score
```

[Learn more](https://docs.microsoft.com/azure/machine-learning/how-to-inference-server-http) about the Azure machine learning inference HTTP server.

### Deploy to ACI container
We are ready to deploy. 

Create an inference configuration which gives specifies the inferencing environment and scripts. 

Create a deployment configuration file to specify the number of CPUs and gigabytes of RAM needed for your ACI container. 

This cell will run for about 7-8 minutes.

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import Webservice
from azureml.core.model import Model

# Write InferenceConfig

inference_config = InferenceConfig(entry_script="pytorch_score.py", environment=myenv)
inference_config.source_directory = "./"


aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=2,
    memory_gb=2,
    tags={"data": "birds", "method": "transfer learning", "framework": "pytorch"},
    description="Classify turkey/chickens using transfer learning with PyTorch",
)

service = Model.deploy(
    workspace=ws,
    name="deploy-aci-pytorch",
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
)

service.wait_for_deployment(True)
print(service.state)

If your deployment fails for any reason and you need to redeploy, make sure to delete the service before you do so: `service.delete()`

**Tip: If something goes wrong with the deployment, the first thing to look at is the logs from the service by running the following command:**

In [ ]:
service.get_logs()

Get the web service's HTTP endpoint, which accepts REST client calls. This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.

In [ ]:
print(service.scoring_uri)

### Test the web service
Finally, let's test our deployed web service. We will send the data as a JSON string to the web service hosted in ACI and use the SDK's `run` API to invoke the service. Here we will take an image from our validation data to predict on.

In [ ]:
import json
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline
plt.imshow(Image.open("test_img.jpg"))

In [ ]:
import torch
from torchvision import transforms


def preprocess(image_file):
    """Preprocess the input image."""
    data_transforms = transforms.Compose(
        [
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )

    image = Image.open(image_file)
    image = data_transforms(image).float()
    image = torch.tensor(image)
    image = image.unsqueeze(0)
    return image.numpy()

In [ ]:
input_data = preprocess("test_img.jpg")
result = service.run(input_data=json.dumps({"data": input_data.tolist()}))
print(result)

## Clean up
Once you no longer need the web service, you can delete it with a simple API call.

In [ ]:
service.delete()